In [1]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import re
from scipy import stats
from pathlib import Path

# directory and file paths
data_dir = Path("../data/")
raw_data_dir = data_dir / "raw"
interim_data_dir = data_dir / "interim"
processed_data_dir = data_dir / "processed"

raw_rental_vacancy_file = raw_data_dir / "rental_vacancy_rates_75.csv"
raw_homeowner_vacancy_file = raw_data_dir / "homeowner_vacancy_rates_75.csv"
raw_homeownership_file = raw_data_dir / "homeownership_rates_75.csv"
raw_covid_cases_file = raw_data_dir / "COVID-19_Case_Surveillance_Public_Use_Data.csv"
raw_geo_covid_cases_file = raw_data_dir / "COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
msa_file = raw_data_dir / "msa.csv"


interim_rental_vacancy_file = interim_data_dir / "rental_vacancy_rates_75.csv"
interim_homeowner_vacancy_file = interim_data_dir / "homeowner_vacancy_rates_75.csv"
interim_homeownership_file = interim_data_dir / "homeownership_rates_75.csv"
population_file = interim_data_dir / "MSA_populations.csv"

# Housing Vacancies and Homeownership (Census Data)

In [2]:
# Since the datasets are stored similarly to pivot tables,
# they need to be flattened in order to manipulate
def flatten_df(df):
    years = ["2020","2019","2018","2017","2016","2015"]
    quarters = ["Q1 ","Q2 ","Q3 ","Q4 "]
    moes = ['Margin of Error1', 'Margin of Error1.1', 'Margin of Error1.2',
       'Margin of Error1.3', 'Margin of Error1.4', 'Margin of Error1.5',
       'Margin of Error1.6', 'Margin of Error1.7', 'Margin of Error1.8',
       'Margin of Error1.9', 'Margin of Error1.10', 'Margin of Error1.11',
       'Margin of Error1.12', 'Margin of Error1.13', 'Margin of Error1.14',
       'Margin of Error1.15', 'Margin of Error2', 'Margin of Error1.16',
       'Margin of Error1.17', 'Margin of Error1.18', 'Margin of Error3',
       'Margin of Error3.1', 'Margin of Error3.2', 'Margin of Error3.3']
    data = []
    for index, row in df.iterrows():
        count = -1
        for j,year in enumerate(years):
            for k,quarter in enumerate(quarters):
                count += 1
                data.append([row["Metropolitan Statistical Area"],
                                quarter,year,
                                row[quarter + year],
                                row[moes[count]]])
    return pd.DataFrame(data=data,columns=['MSA','quarter','year','vacancy rate','margin of error'])

## Rental Vacancy Rates for the 75 Largest MSAs (2015 - 2020)

In [3]:
# import and clean dataset
rental_vacancy_df = pd.read_csv(interim_rental_vacancy_file)

rental_vacancy_df["Metropolitan Statistical Area"] = rental_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
rental_vacancy_df["Metropolitan Statistical Area"] = rental_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
rental_vacancy_df = rental_vacancy_df.replace('(z)', np.NaN, regex=False)
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))

rental_vacancy_df.head()

/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",10.2,9.2,8.1,8.5,3.3,5.3,3.6,5.0,2.0,...,5.2,8.0,3.5,5.1,14.6,9.7,19.9,11.9,10.3,8.9
1,"Albany-Schenectady-Troy, NY",7.9,6.7,6.7,5.6,10.1,7.1,15.3,10.2,7.9,...,3.6,4.6,9.6,6.8,7.9,5.7,5.6,5.1,3.0,3.9
2,"Albuquerque, NM",4.3,2.8,6.9,4,6.8,3.4,4.1,2.5,7.1,...,11.6,4.3,7.3,3.3,7.7,3.3,7.4,3.6,6.4,3.5
3,"Allentown-Bethlehem-Easton, PA-NJ",4.5,5.9,5.6,6.6,2.9,4.2,2.5,4.9,7.7,...,4.6,6.7,3.5,4.6,3.4,5.1,6.0,6.3,2.4,3.7
4,"Atlanta-Sandy Springs-Roswell, GA1",6.9,2.5,5.1,2.3,5.8,2.4,7.7,2.7,7.8,...,6.4,2.6,9.5,2.5,8.4,2.3,7.9,2.3,6.8,2.3


In [4]:
# convert quasi pivot table into flat dataframe
rv_flat_df = flatten_df(rental_vacancy_df)
rv_flat_df = rv_flat_df.rename(columns={"vacancy rate":"rental vacancy rate",
                                        "margin of error":"rental vacancy rate margin of error"})
rv_flat_df.head()

,MSA,quarter,year,rental vacancy rate,rental vacancy rate margin of error
0,"Akron, OH",Q1,2020,10.2,9.2
1,"Akron, OH",Q2,2020,8.1,8.5
2,"Akron, OH",Q3,2020,3.3,5.3
3,"Akron, OH",Q4,2020,3.6,5.0
4,"Akron, OH",Q1,2019,2.0,4.1


## Homeowner Vacancy Rates for the 75 Largest MSAs (2015 - 2020)

In [5]:
# import and clean dataset
homeowner_vacancy_df = pd.read_csv(interim_homeowner_vacancy_file)
homeowner_vacancy_df["Metropolitan Statistical Area"] = homeowner_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
homeowner_vacancy_df["Metropolitan Statistical Area"] = homeowner_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
homeowner_vacancy_df = homeowner_vacancy_df.replace('(z)', np.NaN, regex=False)
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))
homeowner_vacancy_df.head()

/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",0.5,1.4,0.5,1.4,0.5,1.5,NaN,NaN,0.0,...,0.5,1.6,0.0,NaN,0.5,1.3,0.8,1.5,0.5,1.3
1,"Albany-Schenectady-Troy, NY",0.9,1.9,0.9,1.9,5.3,4.2,4.9,4,1.6,...,2.2,2.9,1.3,2,2.8,2.7,1.8,2.1,1.1,1.7
2,"Albuquerque, NM",2,1.3,1.6,1.1,1,1,0.7,0.8,1.9,...,1.5,1.3,2.5,1.6,3.9,2,2.5,1.6,2.1,1.5
3,"Allentown-Bethlehem-Easton, PA-NJ",0.6,1.4,0.5,1.4,0.5,1.4,1.3,2.2,1.2,...,1.8,2.9,0.5,1.2,2.5,2.7,2.1,2.5,2.0,2.6
4,"Atlanta-Sandy Springs-Roswell, GA1",1.2,0.8,1,0.7,0.3,0.4,0.8,0.7,0.9,...,1.5,1,2.9,1.2,2.4,1.1,1.8,0.9,1.7,0.9


In [6]:
# convert quasi pivot table into flat dataframe
hv_flat_df = flatten_df(homeowner_vacancy_df)
hv_flat_df = hv_flat_df.rename(columns={"vacancy rate":"homeowner vacancy rate",
                                        "margin of error":"homeowner vacancy rate margin of error"})
hv_flat_df.head()

,MSA,quarter,year,homeowner vacancy rate,homeowner vacancy rate margin of error
0,"Akron, OH",Q1,2020,0.5,1.4
1,"Akron, OH",Q2,2020,0.5,1.4
2,"Akron, OH",Q3,2020,0.5,1.5
3,"Akron, OH",Q4,2020,NaN,NaN
4,"Akron, OH",Q1,2019,0.0,NaN


## Homeownership Rates for the 75 Largest MSAs (2015 - 2020)

In [7]:
# import and clean dataset
homeownership_df = pd.read_csv(interim_homeownership_file)
homeownership_df["Metropolitan Statistical Area"] = homeownership_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
homeownership_df["Metropolitan Statistical Area"] = homeownership_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
homeownership_df = homeownership_df.replace('(z)', np.NaN, regex=False)
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))
homeownership_df.head()

/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/Users/jamila/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",71.9,9.7,71.7,9.9,68.5,10.1,66.4,9.6,68.3,...,73.2,10.2,68.6,8.7,73.9,8.0,79.9,7.3,73.4,8.4
1,"Albany-Schenectady-Troy, NY",62.9,9.7,57.7,9.6,62.3,9.5,73.1,9.2,60.9,...,60.9,9.2,65.9,8.2,63.5,7.7,66.6,7.5,67.5,7.4
2,"Albuquerque, NM",70.9,4.4,77.2,4.1,66.9,4.7,63.1,4.7,72.1,...,64.7,4.8,63.8,4.6,60.5,4.8,65.9,4.7,67.4,4.7
3,"Allentown-Bethlehem-Easton, PA-NJ",73.1,8.7,69.3,9.5,61.4,9.6,71.9,9.6,70.5,...,68.4,10.2,69.2,7.7,72.7,8.0,71.5,8.0,63.5,8.6
4,"Atlanta-Sandy Springs-Roswell, GA1",63.7,3.8,68.3,3.6,66.6,3.6,67.0,3.7,62.6,...,61.6,3.9,60.5,3.3,59.9,3.2,60.9,3.2,65.5,3.1


In [8]:
# convert quasi pivot table into flat dataframe
homeownership_flat_df = flatten_df(homeownership_df)
homeownership_flat_df = homeownership_flat_df.rename(columns={"vacancy rate":"homeownership rate",
                                                              "margin of error":"homeownership rate margin of error"})
homeownership_flat_df.head()

,MSA,quarter,year,homeownership rate,homeownership rate margin of error
0,"Akron, OH",Q1,2020,71.9,9.7
1,"Akron, OH",Q2,2020,71.7,9.9
2,"Akron, OH",Q3,2020,68.5,10.1
3,"Akron, OH",Q4,2020,66.4,9.6
4,"Akron, OH",Q1,2019,68.3,9.9


In [9]:
housing_df = rv_flat_df.merge(hv_flat_df)
housing_df = housing_df.merge(homeownership_flat_df)
housing_csv = housing_df.to_csv(processed_data_dir/"census_housing.csv",index=False)
housing_df.head()

,MSA,quarter,year,rental vacancy rate,rental vacancy rate margin of error,homeowner vacancy rate,homeowner vacancy rate margin of error,homeownership rate,homeownership rate margin of error
0,"Akron, OH",Q1,2020,10.2,9.2,0.5,1.4,71.9,9.7
1,"Akron, OH",Q2,2020,8.1,8.5,0.5,1.4,71.7,9.9
2,"Akron, OH",Q3,2020,3.3,5.3,0.5,1.5,68.5,10.1
3,"Akron, OH",Q4,2020,3.6,5.0,NaN,NaN,66.4,9.6
4,"Akron, OH",Q1,2019,2.0,4.1,0.0,NaN,68.3,9.9


# COVID-19 Case Surveillance Public Use Data (CDC Data)

In [10]:
# since the covid csv files are quite large, they 
# may need to be split in order to manipulate them
def split_csv(input_file,rows=100000,new_name="output_",output_dir=Path(".")):
    """
    Split a csv file into multiple csv files
    
    Keyword arguments:
        input_file (Path): filepath to csv that will be split
        rows (int): number of rows per output file, default: 100000
        new_name (str): indexed name for output csv files, default: "output_"
        output_dir (Path): filepath to directory for output csv files, default: "."
    """
    in_csv = input_file
    number_lines = sum(1 for row in (open(in_csv))) # total number of lines in csv
    rowsize = rows 

    # start looping through data writing it to a new file for each set
    for i in range(1,number_lines,rowsize):
        df = pd.read_csv(in_csv,
              nrows = rowsize,# number of rows to read at each loop
              skiprows = i)   # skip rows that have been read
        
        # csv to write data to a new file with indexed name - E.g. output_1.csv, output_2.csv, etc.
        out_csv = new_name + str(i//rowsize) + '.csv'
        df.to_csv(output_dir / out_csv,
              index=False,
              header=False,
              mode='a', # append data to csv file
              chunksize=rowsize) #size of data to append for each loop

## COVID-19 Case Surveillance Public Use Data
Learn More: https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf

In [11]:
# import dataset
covid_cases_df = pd.read_csv(raw_covid_cases_file)
covid_cases_df.head()

/Users/jamila/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
0,2020/01/01,2021/01/31,2020/01/01,NaN,Laboratory-confirmed case,Female,0 - 9 Years,Unknown,Missing,Missing,No,Missing
1,2020/01/01,2021/02/02,2020/01/01,NaN,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Missing,Missing,No,Missing
2,2020/01/02,2021/01/27,2020/01/02,NaN,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Missing,Missing,No,Missing
3,2020/01/02,2021/02/02,2020/01/02,NaN,Laboratory-confirmed case,Male,0 - 9 Years,Unknown,Missing,Missing,No,Missing
4,2021/01/01,2020/01/03,2020/01/03,2021/01/01,Laboratory-confirmed case,Male,0 - 9 Years,"White, Non-Hispanic",No,Missing,No,Yes


In [12]:
covid_cases_simple_df = covid_cases_df.drop(columns=["cdc_report_dt","current_status","pos_spec_dt",
                                                    "onset_dt","sex","age_group","medcond_yn"])
covid_cases_simple_df.to_csv(interim_data_dir / "COVID-19_Cases.csv")
covid_cases_simple_df.head()

,cdc_case_earliest_dt,race_ethnicity_combined,hosp_yn,icu_yn,death_yn
0,2020/01/01,Unknown,Missing,Missing,No
1,2020/01/01,Unknown,Missing,Missing,No
2,2020/01/02,Unknown,Missing,Missing,No
3,2020/01/02,Unknown,Missing,Missing,No
4,2021/01/01,"White, Non-Hispanic",No,Missing,No


In [13]:
cases = covid_cases_simple_df
cases["case"] = 1
cases["hosp"] = (cases["hosp_yn"] == "Yes").astype(int)
cases["icu"] = (cases["icu_yn"] == "Yes").astype(int)
cases["death"] = (cases["death_yn"] == "Yes").astype(int)
cases = cases.drop(columns=["hosp_yn","icu_yn","death_yn"]).rename(columns={"cdc_case_earliest_dt ":"case_date"})
cases = cases[["case_date","hosp","icu","death","race_ethnicity_combined","case"]]
cases.to_csv(processed_data_dir / "COVID-19_Cases.csv")
cases.tail(10)

,case_date,hosp,icu,death,race_ethnicity_combined,case
20565335,2020/10/07,0,0,0,"Multiple/Other, Non-Hispanic",1
20565336,2020/10/05,0,0,0,"Multiple/Other, Non-Hispanic",1
20565337,2020/10/07,0,0,0,"Multiple/Other, Non-Hispanic",1
20565338,2020/10/07,0,0,0,"Multiple/Other, Non-Hispanic",1
20565339,2020/10/05,0,0,0,"Multiple/Other, Non-Hispanic",1
20565340,2020/10/03,0,0,0,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1
20565341,2020/10/03,0,0,0,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1
20565342,2020/10/05,0,0,0,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1
20565343,2020/10/02,0,0,0,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1
20565344,2020/09/27,0,0,0,Unknown,1


## COVID-19 Case Surveillance Public Use Data with Geography
Learn More: https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4

In [14]:
geo_covid_cases_df = pd.read_csv(raw_geo_covid_cases_file)
geo_covid_cases_df.head()

/Users/jamila/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing,NaN
1,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
2,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing,NaN
3,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
4,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,NaN


In [15]:
geo_covid_cases_simple_df = geo_covid_cases_df.drop(columns=["age_group","sex","case_positive_specimen_interval",
                                                            "case_onset_interval","process","current_status",
                                                            "symptom_status","underlying_conditions_yn","exposure_yn"
                                                            ])
geo_covid_cases_simple_df.to_csv(interim_data_dir / "COVID-19_Cases_Geo.csv")
geo_covid_cases_simple_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,race,ethnicity,hosp_yn,icu_yn,death_yn
0,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,No,Missing,Missing
1,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Missing
2,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,No,Missing,Missing
3,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Missing
4,2020-01,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Missing


In [16]:
geo = geo_covid_cases_simple_df
geo["case"] = 1
geo["hosp"] = (geo["hosp_yn"] == "Yes").astype(int)
geo["icu"] = (geo["icu_yn"] == "Yes").astype(int)
geo["death"] = (geo["death_yn"] == "Yes").astype(int)
geo = geo.drop(columns=["hosp_yn","icu_yn","death_yn"])
geo.tail()

,case_month,res_state,state_fips_code,res_county,county_fips_code,race,ethnicity,case,hosp,icu,death
21950109,2020-12,AZ,4.0,YUMA,4027.0,White,Non-Hispanic/Latino,1,0,0,1
21950110,2020-12,AZ,4.0,YUMA,4027.0,White,Non-Hispanic/Latino,1,1,0,1
21950111,2020-12,AZ,4.0,YUMA,4027.0,White,Non-Hispanic/Latino,1,1,0,1
21950112,2020-12,AZ,4.0,YUMA,4027.0,White,Non-Hispanic/Latino,1,1,0,1
21950113,2020-12,AZ,4.0,YUMA,4027.0,White,Non-Hispanic/Latino,1,1,0,1


In [17]:
msa = pd.read_csv(msa_file)
msa["county_fips_code"] = msa["fipsstatecode"]*1000 + msa["fipscountycode"]
msa = msa[["cbsacode","county_fips_code","statename","countycountyequivalent","cbsatitle"]].dropna()
geo_msa = geo.merge(msa,how="left",on="county_fips_code")
geo_msa["cbsacode"] = geo_msa["cbsacode"].fillna(0).astype(int)
geo_msa["county_fips_code"] = geo_msa["county_fips_code"].fillna(0).astype(int)
geo_msa = geo_msa[["case_month","hosp","icu","death","race","ethnicity","cbsatitle", "cbsacode",
                  "county_fips_code","case"]]
geo_msa = geo_msa.rename(columns={"cbsatitle":"cbsa_title","cbsacode":"cbsa_code"})

In [18]:
geo_msa.to_csv(processed_data_dir / "COVID-19_Cases_Geo.csv",index=False)
geo_msa.tail(10)

,case_month,hosp,icu,death,race,ethnicity,cbsa_title,cbsa_code,county_fips_code,case
21950104,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950105,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950106,2020-12,0,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950107,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950108,2020-12,0,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950109,2020-12,0,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950110,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950111,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950112,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1
21950113,2020-12,1,0,1,White,Non-Hispanic/Latino,"Yuma, AZ",49740,4027,1


## CBSA (MSA) Population Data
Learn More: https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-metro-and-micro-statistical-areas.html

In [26]:
# Process US population file
pop_df = pd.read_csv(population_file)
pop_df = pop_df.rename(columns={"CBSA":"cbsa_code","POPESTIMATE2019":"population"})
pop_df = pop_df[(pop_df["LSAD"]=="Metropolitan Statistical Area")|(pop_df["LSAD"]=="Micropolitan Statistical Area")]
pop_df = pop_df.drop(columns=["STCOU","LSAD"])

In [27]:
# Group cases by month and MSA
geo_df = geo_msa.groupby(["case_month","cbsa_title","cbsa_code"]).sum().drop(columns=["county_fips_code"])
geo_df.reset_index(inplace=True)

In [28]:
# Merge population DataFrame with Covid-19 Cases w/ Geo DataFrame
geo_pop = geo_df.merge(pop_df,on="cbsa_code",how="left")
geo_pop["cases_per_capita"] = geo_pop["case"] / geo_pop["population"]
geo_pop = geo_pop.rename(columns={"case":"cases","death":"deaths"})
geo_pop = geo_pop[["case_month","cbsa_title","cbsa_code","cases","hosp",
                  "icu","deaths","cases_per_capita","population"]]
geo_pop.to_csv(processed_data_dir / "Monthly_COVID-19_Cases_By_MSA.csv",index=False)
geo_pop.tail(40)

,case_month,cbsa_title,cbsa_code,cases,hosp,icu,deaths,cases_per_capita,population
8874,2021-03,"Van Wert, OH",46780,11,0,0,0,0.000389,28275.0
8875,2021-03,"Vernal, UT",46860,96,0,0,0,0.002687,35734.0
8876,2021-03,"Vidalia, GA",47080,11,0,0,0,0.000306,36002.0
8877,2021-03,"Vineland-Bridgeton, NJ",47220,232,14,0,0,0.001552,149527.0
8878,2021-03,"Virginia Beach-Norfolk-Newport News, VA-NC",47260,23,0,0,0,0.000013,1768901.0
8879,2021-03,"Visalia-Porterville, CA",47300,293,8,0,0,0.000628,466195.0
8880,2021-03,"Walla Walla, WA",47460,14,1,0,0,0.000230,60760.0
8881,2021-03,"Wapakoneta, OH",47540,12,0,0,0,0.000263,45656.0
8882,2021-03,"Warner Robins, GA",47580,91,0,0,0,0.000491,185409.0
8883,2021-03,"Warren, PA",47620,37,1,0,0,0.000944,39191.0


## Split CSV

In [22]:
# split_csv(interim_data_dir / "COVID-19_Cases.csv",rows=1000000,
#           new_name="COVID-19_Cases_",output_dir=processed_data_dir)
# split_csv(interim_data_dir / "COVID-19_Cases_Geo.csv",rows=1000000,
#           new_name="COVID-19_Cases_Geo_",output_dir=processed_data_dir)